In [546]:
import subprocess
subprocess.run(["sudo","cpupower","frequency-set","-u","6.0GHz"],stderr=subprocess.DEVNULL,stdout=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '6.0GHz'], returncode=0)

In [547]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,r2_score
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [548]:
from sklearn.datasets import load_iris
iris=load_iris()
df=pd.DataFrame(data=iris.data,columns=iris.feature_names)
df['target']=iris.target
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [549]:
df2=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Iris%20Species/Iris.csv")

In [550]:
df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [551]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [552]:
categorical_cols=['target']

In [553]:
train_df,test_df=train_test_split(df,test_size=0.25,random_state=42)

In [554]:
label_encoder={}

for col in categorical_cols:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col])
    test_df[col]=le.transform(test_df[col])
    label_encoder=le

In [555]:
print(label_encoder.classes_)

[0 1 2]


In [556]:
X_train_df=train_df.drop(columns=['target'])
y_train_df=train_df['target']
X_test_df=test_df.drop(columns=['target'])
y_test_df=test_df['target']

In [557]:
X_test_df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
73,6.1,2.8,4.7,1.2
18,5.7,3.8,1.7,0.3
118,7.7,2.6,6.9,2.3
78,6.0,2.9,4.5,1.5
76,6.8,2.8,4.8,1.4


In [558]:
y_train_df.head(5)

4      0
32     0
142    2
85     1
86     1
Name: target, dtype: int64

In [559]:
import torch


In [560]:
scaler_X=StandardScaler()
scaler_Y=StandardScaler()
X_train_scaler=scaler_X.fit_transform(X_train_df)
X_test_scaler=scaler_X.transform(X_test_df)
# y_train_scaler=scaler_Y.fit_transform(y_train_df)
# y_test_scaler=scaler_Y.transform(y_test_df)

In [561]:
X_train_tensor=torch.tensor(X_train_scaler,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test_scaler,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train_df.values,dtype=torch.long)
y_test_tensor=torch.tensor(y_test_df.values,dtype=torch.long)

In [562]:
import torch.nn as nn
import torch.optim as optim


In [563]:
class IrisClassifierModel(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(IrisClassifierModel,self).__init__()
        self.network=nn.Sequential(
            nn.Linear(input_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,output_size)
        )
    def forward(self,x):
        return self.network(x)

In [564]:
input_size=X_train_tensor.shape[1]
hidden_size=16
output_size=3


In [565]:
model=IrisClassifierModel(input_size,hidden_size,output_size)
criterion=nn.CrossEntropyLoss()
lr=0.001
optimizer=optim.Adam(model.parameters())

In [566]:
print(X_train_tensor.shape,y_train_tensor.shape)
print(y_train_tensor.T)

torch.Size([112, 4]) torch.Size([112])
tensor([0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0,
        0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0,
        2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1,
        1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2,
        0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])


In [567]:
epochs=500
train_losses=[]
valid_losses=[]
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    train_predictions=model(X_train_tensor)
    train_loss=criterion(train_predictions,y_train_tensor)
    train_loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        valid_prediction=model(X_test_tensor)
        valid_loss=criterion(valid_prediction,y_test_tensor)
        valid_losses.append(valid_loss)
        train_losses.append(train_loss)
    if (epoch+1)% 100==0:
        print(f"Epoach:{epoch+1}/{epochs},loss:{train_loss.item():.4f}")

Epoach:100/500,loss:0.5239
Epoach:200/500,loss:0.2039
Epoach:300/500,loss:0.0946
Epoach:400/500,loss:0.0654
Epoach:500/500,loss:0.0557


In [568]:
model.eval()
with torch.no_grad():
    y_pred=model(X_test_tensor)
    y_pred_lebels=torch.argmax(y_pred,dim=1)
    accuracy=(y_pred_lebels==y_test_tensor).sum().item()/y_test_tensor.size(0)
    print(f"Test accuracy: {accuracy}")

Test accuracy: 1.0


In [ ]:
def predict_iris(sepal_length,sepal_width,petal_length,petal_width):
    input_df=pd.DataFrame(data=[[sepal_length,sepal_width,petal_length,petal_width]],columns=['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)'])
    input_df=scaler_X.transform(input_df)
    input_tensor=torch.tensor(input_df,dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        results=model(input_tensor)
        result=torch.argmax(results,dim=1).item()
        result=iris.target_names[result]
    print(result)
    

In [570]:
predict_iris(5.1,3.5,1.4,0.2)

setosa


In [571]:
subprocess.run(["sudo","cpupower","frequency-set","-u","2.8GHz"],stderr=subprocess.DEVNULL,stdout=subprocess.DEVNULL)

CompletedProcess(args=['sudo', 'cpupower', 'frequency-set', '-u', '2.8GHz'], returncode=0)